If you see this, it's successfully commited

In [ ]:
!pip install pyspark

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
path = "/content/drive/MyDrive/Yandex Big Data Colab/data/"

shows_log = spark.sparkContext.textFile(path + "shows-log2")
clicks_log = spark.sparkContext.textFile(path + "clicks-log")
banners_dict = spark.sparkContext.textFile(path + "banners_dict")
banner_info = spark.sparkContext.textFile(path + "banner_info")
user_profile = spark.sparkContext.textFile(path + "user_profile")

shows_log = spark.sparkContext.textFile("/content/drive/MyDrive/Yandex Big Data Colab/shows-log")

In [ ]:
for i in user_profile.collect():
    print(i)

In [158]:
from types import LambdaType
import json
from pyspark.sql.functions import lit, udf, col
from pyspark.sql.types import StringType
from pyspark.sql import Row

def from_data_to_rdd(symbol="\t"):
    def f(data):
        return tuple(data.split(symbol))
    return f

def convert_json_to_dict(x):
    res = json.loads(x)
    res["uid"] = str(res["uid"])
    return res

def convert_json_to_rdd(rdd):
    return rdd.map(convert_json_to_dict).map(lambda x: tuple(i for i in x.values()))

def get_revenue(event1, event2, revenue):
    if event1 == event2:
        return revenue
    return "0"

def join_with_tabs(x):
    return "\t".join(x)

def get_fixNone(n):
    def f(x):
        if x[0] == None:
            x = ("None",)+(x[1],)
        if x[1][1] == None:
            if n == 1:
                x = (x[0],)+((x[1][0], "None"),)
            else:
                x = (x[0],)+((x[1][0], ("None",)*n),)
        return x
    return f

def process(shows, clicks, banners_dict, banner_info, user_profile):
    # convert to rdd
    clicks = clicks.map(from_data_to_rdd()) # dt, uid, banner_id
    banners_dict = banners_dict.map(from_data_to_rdd()) # banner_id, banner_type
    shows = shows.map(from_data_to_rdd()) # dt, uid, banner_id, banner_type
    banner_info = banner_info.map(from_data_to_rdd(",")) # banner_type, pay_event_type, currency, cost
    user_profile = convert_json_to_rdd(user_profile) # age, uid, gender

    # step 1
    clicks = clicks.map(lambda x: (x[2], (x[1], x[0]))) # banner_id, (uid, dt)
    clicks_typed = clicks.leftOuterJoin(banners_dict) # banner_id, ((uid, dt), banner_type)
    clicks_typed = clicks_typed.map(get_fixNone(1))

    # step 2
    clicks_typed = clicks_typed.map(lambda x: (x[1][1], (*x[1][0], x[0], "click"))) # banner_type, (uid, dt, banner_id, event_type)
    shows = shows.map(lambda x: (x[3], (x[1], x[0], x[2], "show"))) # banner_type, (uid, dt, banner_id, event_type)
    union = clicks_typed.union(shows)
    #union = shows

    # step 3
    banner_info = banner_info.map(lambda x: (x[0], (*x[1:],))) # banner_type, (pay_event_type, currency, cost)
    union_revenue = union.leftOuterJoin(banner_info) # banner_type, ((uid, dt, banner_id, event_type), (pay_event_type, currency, cost))
    union_revenue = union_revenue.map(get_fixNone(3))
    union_revenue = union_revenue.map(lambda x:(
                                      x[1][0][0], (*x[1][0][1:], *x[1][1], x[0], get_revenue( x[1][0][3], x[1][1][0], x[1][1][2]))
                                       )) # uid, (dt, banner_id, event_type,  pay_event_type, currency, cost,  banner_type,  revenue)

    # step 4
    user_profile = user_profile.map(lambda x: (x[1], (x[0], x[2]))) # uid, (age, gender)
    res = union_revenue.leftOuterJoin(user_profile) # uid, ((dt, banner_id, event_type, pay_event_type, currency, cost, banner_type, revenue), (age, gender))
    res = res.map(get_fixNone(2))
    # convert result
    res = res.map(lambda x: (
                  x[1][0][0], x[1][0][2], x[0], x[1][0][1], x[1][0][6], *x[1][1], *x[1][0][3:5], x[1][0][7]
                  )) # dt, event_type, uid, banner_id, banner_type, age, gender, pay_event_type, currency, revenue
    res = res.map(join_with_tabs)

    return res # res.groupBy(lambda x: x).map(lambda x: x[0])


In [ ]:
result = process(shows_log, clicks_log, banners_dict, banner_info, user_profile)
print(result)

In [ ]:
t = False
for i in result.collect():
    print(i)

In [ ]:
spark.stop()